# Data Processing

In [ ]:
import pandas as pd
from sklearn import preprocessing

import plotly.offline as py
import plotly.express as px
py.init_notebook_mode(connected=True)

In [ ]:
df = pd.read_csv('../data/trivago/train.csv')
df_meta = pd.read_csv("../data/trivago/item_metadata.csv")

In [ ]:
df.action_type.value_counts()

In [ ]:
df.head()

In [ ]:
df[df["action_type"] == "clickout item"]

In [ ]:
RECSYS_CITIES = [
    "Lausanne, Switzerland",
    "New York, USA",
    "Barcelona, Spain",
    "Chicago, USA",
    "Dublin, Ireland",
    "Hong Kong, Hong Kong",
    "Vienna, Austria",
    "Boston, USA",
    "Como, Italy",
    "Vancouver, Canada",
    "Copenhagen, Denmark",
    "Rio de Janeiro, Brazil",
]

In [ ]:
df = df[(df["action_type"] == "clickout item")]
df = df[df["city"].isin(RECSYS_CITIES)]
df = df[["timestamp", "user_id", "reference", "impressions"]]

In [ ]:
df = df.sort_values("timestamp") 

In [ ]:
df = df.rename(columns={"reference": "item_id"})
df

In [ ]:
df["impressions"] = df["impressions"].apply(lambda x: x.split("|"))
df

In [ ]:
df["item_id"] = df["item_id"].astype("int")
df["impressions"] = df["impressions"].apply(lambda x: [int(i) for i in x])

In [ ]:
df["pos"] = df[["item_id", "impressions"]].apply(lambda x: x.impressions.index(x.item_id) if x.item_id in x.impressions else -1, axis=1)
df = df.drop(df[df["pos"] == -1].index)

In [ ]:
df = df.explode("impressions")
df["clicked"] = df[["impressions", "item_id"]].apply(lambda x: 1 if int(x["impressions"]) == int(x["item_id"]) else 0, axis=1)
df

In [ ]:
df = df[["user_id", "impressions", "clicked", "pos"]]
df = df.rename(columns={"impressions": "item_id"})

In [ ]:
item_encoder = preprocessing.LabelEncoder().fit(df.item_id.values)
df.item_id = item_encoder.transform(df.item_id.values)

In [ ]:
user_encoder = preprocessing.LabelEncoder().fit(df.user_id.values)
df.user_id = user_encoder.transform(df.user_id.values)

In [ ]:
df.clicked.value_counts()

In [ ]:
df = df[["user_id", "item_id", "clicked"]].groupby(["user_id", "item_id"]).max().reset_index()
df["clicked"].value_counts()

In [ ]:
df.to_csv("../data/trivago/train_processed.csv", index=False)

# TRAIN PMF

In [1]:
from __future__ import print_function

import os
import pickle

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data

import matplotlib.pyplot as plt

import sys 
sys.path.append('..')

from src.model.pmf import PMF

In [2]:
def RMSE(preds, truth):
    return np.sqrt(np.mean(np.square(preds-truth)))

In [3]:
batch_size = 1000
epoches = 10000
no_cuda = False
seed = 1
weight_decay = 0.1
embedding_feature_size = 50
ratio = 0.8
lr = 0.0001
momentum = 0.9

In [4]:
df = pd.read_csv("../data/trivago/train_processed.csv")
df.clicked.value_counts()

0    804267
1     51676
Name: clicked, dtype: int64

In [ ]:
df["clicked_norm"] = df["clicked"].apply(lambda x: 1 if x == 1 else -1)

In [ ]:
from sklearn.utils import resample

pos_click = df[df["clicked"] == 1]
neg_click = df[df["clicked"] == 0]

neg_upsample = resample(neg_click,
             replace=True,
             n_samples=len(pos_click),
             random_state=42)

print(neg_upsample.shape)

data_upsampled = pd.concat([pos_click, neg_upsample])
data_upsampled.clicked.value_counts()

In [ ]:
data = data_upsampled[["user_id", "item_id", "clicked_norm"]].values

In [5]:
NUM_ITEMS = df.item_id.max() + 1
NUM_USERS = df.user_id.max() + 1

print(NUM_USERS, NUM_ITEMS)

26306 11972


In [ ]:
# Split data
train_data = data[:int(ratio * data.shape[0])]
vali_data = data[int(ratio * data.shape[0]): int((ratio+(1-ratio)/2)*data.shape[0])]
test_data = data[int((ratio + (1 - ratio) / 2) * data.shape[0]) :]

In [ ]:
no_cuda=False

# Get CUDA device if available
cuda = torch.cuda.is_available()
 
# Set device to CUDA or CPU, depending on availability and desire
device = torch.device("cuda" if cuda and no_cuda else "cpu")
 
# Generate and apply seeds
torch.manual_seed(seed=seed)
if cuda:
    torch.cuda.empty_cache()
    torch.cuda.manual_seed(seed=seed)
 
# Specify number of workers for cuda
kwargs = {"num_workers":1, "pin_memory":True} if cuda else {}
 
# Construct Data Loaders
train_data_loader = torch.utils.data.DataLoader(torch.from_numpy(train_data), batch_size=batch_size, shuffle=False, **kwargs)
test_data_loader = torch.utils.data.DataLoader(torch.from_numpy(test_data), batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:
# Initialize model
model = PMF(n_users=NUM_USERS, n_items=NUM_ITEMS, n_factors=embedding_feature_size, no_cuda=no_cuda)
 
# Move model to CUDA if CUDA selected
if cuda and not no_cuda:
    model.cuda()
    print("Model moved to CUDA")
 
# Set loss function
loss_function = nn.MSELoss(reduction="sum")

# Set optimizer (uncomment Adam for adam)
# optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
# Function for training one epoch
def train(epoch, train_data_loader):
    # Initialize
    model.train()
    epoch_loss = 0.0
    optimizer.zero_grad()
 
    # Go through batches
    for batch_idx, ele in enumerate(train_data_loader):
        # Zero optimizer gradient
        optimizer.zero_grad()
 
        # Extract user_id_nums: row 0, item_id_nums: col 1 , ratings: val 2
        row = ele[:, 0]
        col = ele[:, 1]
        val = ele[:, 2]
 
        # Set to variables
        row = Variable(row.long())
        if isinstance(col, list):
            col = tuple(Variable(c.long()) for c in col)
        else:
            col = Variable(col.long())
        val = Variable(val.float())

        # Move data to CUDA
        if cuda and not no_cuda:
            row = row.cuda()
            col = col.cuda()
            val = val.cuda()
 
        # Train
        preds = model.forward(row, col)
        loss = loss_function(preds, val)
        loss.backward()
        optimizer.step()
 
        # Update epoch loss
        epoch_loss += loss.data
 
    epoch_loss /= train_data_loader.dataset.shape[0]
    return epoch_loss

In [ ]:
# Training Model
count = 0
train_loss_list = []
last_vali_rmse = None
train_rmse_list = []
vali_rmse_list = []
print("parameters are: train ratio:{:f},batch_size:{:d}, epoches:{:d}, weight_decay:{:f}".format(ratio, batch_size, epoches, weight_decay))
print(model)

# Go through epochs
for epoch in range(1, epoches+1):

    # Train epoch
    train_epoch_loss = train(epoch, train_data_loader)

    # Get epoch loss
    train_loss_list.append(train_epoch_loss.cpu())

    # Move validation data to CUDA
    if cuda and not no_cuda:
        vali_row = Variable(torch.from_numpy(vali_data[:, 0]).long()).cuda()
        vali_col = Variable(torch.from_numpy(vali_data[:, 1]).long()).cuda()
    else:
        vali_row = Variable(torch.from_numpy(vali_data[:, 0]).long())
        vali_col = Variable(torch.from_numpy(vali_data[:, 1]).long())

    # Get validation predictions
    vali_preds = model.predict(vali_row, vali_col)

    # Calculate train rmse loss
    train_rmse = np.sqrt(train_epoch_loss.cpu())

    # Calculate validation rmse loss
    if cuda and not no_cuda:
        vali_rmse = RMSE(vali_preds.cpu().data.numpy(), vali_data[:, 2])
    else:
        vali_rmse = RMSE(vali_preds.data.numpy(), vali_data[:, 2])

    # Add losses to rmse loss lists
    train_rmse_list.append(train_rmse)
    vali_rmse_list.append(vali_rmse)

    print("Training epoch:{: d}, training rmse:{: .6f}, vali rmse:{:.6f}". \
            format(epoch, train_rmse, vali_rmse))

    # Early stop condition
    if last_vali_rmse and last_vali_rmse < vali_rmse:
        break
    else:
        last_vali_rmse = vali_rmse
    

In [ ]:
# Testing Model

# Move test set to CUDA
if cuda:
    test_row = Variable(torch.from_numpy(test_data[:, 0]).long()).cuda()
    test_col = Variable(torch.from_numpy(test_data[:, 1]).long()).cuda()
else:
    test_row = Variable(torch.from_numpy(test_data[:, 0]).long())
    test_col = Variable(torch.from_numpy(test_data[:, 1]).long())
 
# Get test predictions
preds = model.predict(test_row, test_col)
 
# Get test rmse loss
if cuda:
    test_rmse = RMSE(preds.cpu().data.numpy(), test_data[:, 2])
else:
    test_rmse = RMSE(preds.data.numpy(), test_data[:, 2])
print("Test rmse: {:f}".format(test_rmse))

In [ ]:
# Create plots
plt.figure(1)
plt.plot(range(1, len(train_rmse_list)+1), train_rmse_list, color="r", label="train rmse")
plt.plot(range(1, len(vali_rmse_list)+1), vali_rmse_list, color="b", label="test rmse")
plt.legend()
plt.annotate(r"train=%f" % (train_rmse_list[-1]), xy=(len(train_rmse_list), train_rmse_list[-1]),
             xycoords="data", xytext=(-30, 30), textcoords="offset points", fontsize=10,
             arrowprops=dict(arrowstyle="->", connectionstyle="arc3, rad=.2"))
plt.annotate(r"vali=%f" % (vali_rmse_list[-1]), xy=(len(vali_rmse_list), vali_rmse_list[-1]),
             xycoords="data", xytext=(-30, 30), textcoords="offset points", fontsize=10,
             arrowprops=dict(arrowstyle="->", connectionstyle="arc3, rad=.2"))
plt.xlim([1, len(train_rmse_list)+10])
plt.xlabel("iterations")
plt.ylabel("RMSE")
plt.title("RMSE Curve in Training Process")
plt.show()

In [ ]:
# Save model
path_to_trained_pmf = "../model/pmf/trivago_emb_{:d}_ratio_{:f}_bs_{:d}_e_{:d}_wd_{:f}_lr_{:f}_trained_pmf.pt".format(embedding_feature_size, ratio, batch_size, len(train_rmse_list), weight_decay, lr)
torch.save(model.state_dict(), path_to_trained_pmf)

In [ ]:
idx =  0
(model.predict(
    torch.tensor([data[idx][0]]).long().to("cuda"), 
    torch.tensor([data[idx][1]]).long().to("cuda")
).cpu().data[0] + 1) / 2, (data[idx][2] + 1) / 2

# Data Processing

In [ ]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
import plotly.offline as py
import plotly.express as px
py.init_notebook_mode(connected=True)

In [ ]:
df = pd.read_csv("../data/trivago/train_processed.csv")
df.clicked.value_counts()

In [ ]:
df.head()

In [ ]:
users_dict = {user: [] for user in set(df["user_id"])}

ratings_df_gen = df.iterrows()
users_dict_positive_items = {
    user: [] for user in set(df["user_id"])
}
for data in ratings_df_gen:
    users_dict[data[1]["user_id"]].append(
        (data[1]["item_id"], data[1]["clicked"])
    )
    if data[1]["clicked"] > 0:
        users_dict_positive_items[data[1]["user_id"]].append(
            (data[1]["item_id"], data[1]["clicked"])
        )
users_history_lens = [
    len(users_dict_positive_items[u])
    for u in set(df["user_id"])
]

users_num = max(df["user_id"]) + 1
items_num = max(df["item_id"]) + 1

print(users_num, items_num)

In [ ]:
train_users_num = int(users_num * 0.8)
train_users_dict = {k: users_dict.get(k) for k in range(0, train_users_num - 1)}
train_users_history_lens = users_history_lens[:train_users_num]

# Evaluating setting
eval_users_num = int(users_num * 0.2)
eval_users_dict = {
    k: users_dict[k] for k in range(users_num - eval_users_num, users_num)
}
eval_users_history_lens = users_history_lens[-eval_users_num:]


In [ ]:
# Save processed data
with open("../data/trivago/train_users_dict.pkl", "wb") as file:
    pickle.dump(train_users_dict, file)

with open("../data/trivago/train_users_history_lens.pkl", "wb") as file:
    pickle.dump(train_users_history_lens, file)

with open("../data/trivago/eval_users_dict.pkl", "wb") as file:
    pickle.dump(eval_users_dict, file)

with open("../data/trivago/eval_users_history_lens.pkl", "wb") as file:
    pickle.dump(eval_users_history_lens, file)

with open("../data/trivago/users_history_lens.pkl", "wb") as file:
    pickle.dump(users_history_lens, file)


In [ ]:
z = np.random.geometric(p=0.35, size=items_num)
w = z%10 
w = [i if i > 0 else 10 for i in w]

In [ ]:
px.histogram(w)

In [ ]:
item_group = {i: w[i] for i in range(items_num)}

In [ ]:
with open("../data/trivago/item_groups.pkl", "wb") as file:
    pickle.dump(item_group, file)

In [ ]:
dataset = {}
with open("../data/trivago/train_users_dict.pkl", "rb") as pkl_file:
    dataset["train_users_dict"] = pickle.load(pkl_file)

with open("../data/trivago/train_users_history_lens.pkl", "rb") as pkl_file:
    dataset["train_users_history_lens"] = pickle.load(pkl_file)

with open("../data/trivago/eval_users_dict.pkl", "rb") as pkl_file:
    dataset["eval_users_dict"] = pickle.load(pkl_file)

with open("../data/trivago/eval_users_history_lens.pkl", "rb") as pkl_file:
    dataset["eval_users_history_lens"] = pickle.load(pkl_file)

with open("../data/trivago/users_history_lens.pkl", "rb") as pkl_file:
    dataset["users_history_lens"] = pickle.load(pkl_file)

with open("../data/trivago/item_groups.pkl", "rb") as pkl_file:
    dataset["item_groups"] = pickle.load(pkl_file)



In [ ]:
px.histogram(dataset["train_users_history_lens"])

In [ ]:
available_users = []
for u in dataset["train_users_dict"].keys():
    positive_items = [data[0] for data in dataset["train_users_dict"][u] if data[1] > 0]
    if len(positive_items) > 1:
        available_users.append(u)
len(available_users)